## Setting up the environment

In [ ]:
# move to the project directory
import os
os.chdir("..")

from pathlib import Path
MOLECULES_FILE = 'molecules.json'
MOLECULES_PATH = Path('.', 'data', MOLECULES_FILE)

import pandas as pd
from pprint import pprint

# import the project
from quantum_pipeline.drivers import molecule_loader

In [ ]:
molecules = molecule_loader.load_molecule(MOLECULES_PATH)
pprint(molecules)

### Using the VQERunner programatically

In [ ]:
from quantum_pipeline.runners.vqe_runner import VQERunner

backend = VQERunner.default_backend()
runner = VQERunner(
    filepath=MOLECULES_PATH,
    basis_set='sto3g',
    max_iterations=1,
    convergence_threshold=1e-6,
    optimizer='COBYLA',
    ansatz_reps=3
)
runner.run(backend)

### See results from the IBM Quantum

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService
import os

service = QiskitRuntimeService(
    channel=os.getenv('IBM_RUNTIME_CHANNEL'),
    instance=os.getenv('IBM_RUNTIME_INSTANCE'),
    token=os.getenv('IBM_RUNTIME_TOKEN')
)
job = service.job('cy5seka01rbg008j62mg')
job_result = job.result()

for idx, pub_result in enumerate(job_result):
    print(f"Expectation values for pub {idx}: {pub_result.data.evs}")

### Example Kafka Consumer to receive messages from the pipeline

In [ ]:
from kafka import KafkaConsumer
from quantum_pipeline.stream.serialization.interfaces.vqe import VQEDecoratedResultInterface

class KafkaMessageConsumer:
    def __init__(self, topic='vqe_results', bootstrap_servers='localhost:9092'):
        self.deserializer = VQEDecoratedResultInterface()
        self.consumer = KafkaConsumer(
            topic,
            bootstrap_servers=bootstrap_servers,
            value_deserializer=self.deserializer.from_avro_bytes,
            auto_offset_reset='earliest',
            enable_auto_commit=True,
            group_id='vqe_consumer_group'
        )
    
    def consume_messages(self):
        try:
            for message in self.consumer:
                try:
                    # Process the message
                    decoded_message = message.value
                    yield decoded_message
                except Exception as e:
                    print(f"Error processing message: {str(e)}")
                    continue
        except Exception as e:
            print(f"Error in consumer: {str(e)}")
        finally:
            self.consumer.close()

In [ ]:
consumer = KafkaMessageConsumer()
for msg in consumer.consume_messages():
    print(f"Received message: {msg}")